In [30]:
import sqlite3
from sqlite3 import Error
from sqlite3 import IntegrityError
from ipyfilechooser import FileChooser
import pandas as pd
import io
import requests
import numpy
from scipy import stats
import math

In [3]:
starting_directory = '/Users/Joseph'
chooser = FileChooser(starting_directory)
display(chooser)

FileChooser(path='/Users/Joseph', filename='', title='', show_hidden=False, select_desc='Select', change_desc=…

In [8]:
file=chooser.selected
folder_path=chooser.selected_path
print(file)
print(folder_path)

/Users/Joseph/Desktop/NRSC510B/new_tap_output.csv
/Users/Joseph/Desktop/NRSC510B


In [5]:
baseline_output = pd.read_csv(file, index_col=0)
print(baseline_output)
# X=baseline_output.drop(columns=['Time','n','Number','Tap','Puff','x','y','plate','dataset','Gene','Allele'])
# print(X)

            Time    n  Number  Instantaneous Speed  Interval Speed   Bias  \
0        490.055   56      39               0.0471          0.0548  0.256   
1        490.135   56      39               0.0507          0.0565  0.256   
2        490.212   56      39               0.0503          0.0546  0.256   
3        490.292   56      39               0.0450          0.0481  0.256   
4        490.378   56      39               0.0439          0.0455  0.179   
...          ...  ...     ...                  ...             ...    ...   
1104538  589.610  113      82               0.0830          0.0923  0.241   
1104539  589.684  113      82               0.0790          0.0862  0.215   
1104540  589.771  113      82               0.0748          0.0825  0.228   
1104541  589.851  113      82               0.0757          0.0788  0.291   
1104542  589.932  112      80               0.0734          0.0761  0.338   

          Width  Length      Area  Angular Speed  Aspect Ratio  Kink  Curve

In [9]:
tap_output = pd.read_csv(file, index_col=0)
print(tap_output)

         time  dura   dist      prob     speed  plate      Date Plate_id  \
0     599.959  3.21  0.581  0.803922  0.180997      1  20230407  B0407ba   
1     609.960  2.54  0.509  0.648148  0.200394      1  20230407  B0407ba   
2     619.914  1.84  0.409  0.750000  0.222283      1  20230407  B0407ba   
3     629.897  1.73  0.364  0.474576  0.210405      1  20230407  B0407ba   
4     639.989  1.21  0.247  0.536232  0.204132      1  20230407  B0407ba   
..        ...   ...    ...       ...       ...    ...       ...      ...   
334   859.979  0.79  0.163  0.450000  0.206329     11  20221014  A1014ce   
335   869.983  1.04  0.203  0.350877  0.195192     11  20221014  A1014ce   
336   879.979  0.82  0.163  0.456140  0.198780     11  20221014  A1014ce   
337   889.911  0.72  0.133  0.415094  0.184722     11  20221014  A1014ce   
338  1189.993  3.23  0.687  0.583333  0.212693     11  20221014  A1014ce   

        Screen  taps         dataset      Gene  Allele  
0    PD_Screen   1.0  F32D8.13

In [10]:
PD_tap=tap_output
PD_first_tap = PD_tap[(PD_tap.taps==1)].reset_index().drop(columns="index")
PD_last_three_taps = PD_tap[((PD_tap.taps>=28)&(PD_tap.taps<=30))].reset_index().drop(columns="index")
PD_recov_taps = PD_tap[(PD_tap.taps==31)].reset_index().drop(columns="index")
# print(PD_first_tap)
# # print(PD_last_three_taps)
# # # print(PD_recov_taps)
PD_final_taps=PD_last_three_taps.groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"]).mean().reset_index()
# print(PD_final_taps)
PD_final_taps=PD_final_taps.rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")
PD_first_tap=PD_first_tap.rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
# print(PD_final_taps)
# print(PD_first_tap)
PD_habit_levels = pd.merge(PD_first_tap, PD_final_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
PD_habit_levels = PD_habit_levels.dropna()
# print(PD_habit_levels)
PD_habit_levels['habit_dura']=PD_habit_levels.init_dura-PD_habit_levels.final_dura
PD_habit_levels['habit_prob']=PD_habit_levels.init_prob-PD_habit_levels.final_prob
PD_habit_levels['habit_speed']=PD_habit_levels.init_speed-PD_habit_levels.final_speed
# PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y', 'Allele','Gene'])
PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y'])
PD_recov_taps=PD_recov_taps.rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
PD_habituation = PD_habituation.drop(columns=['taps_x','taps_y','taps','time'])
PD_habituation = PD_habituation.dropna()
# print(PD_habituation)
PD_habituation['recovery_dura']=PD_habituation.init_dura-PD_habituation.recov_dura
PD_habituation['recovery_prob']=PD_habituation.init_prob-PD_habituation.recov_prob
PD_habituation['recovery_speed']=PD_habituation.init_speed-PD_habituation.recov_speed
tap_data=PD_habituation.dropna()
print(tap_data)

     init_dura  init_prob  init_speed  plate      Date Plate_id     Screen  \
0         3.21   0.803922    0.180997      1  20230407  B0407ba  PD_Screen   
1         2.48   0.792453    0.202419      2  20230407  B0407bd  PD_Screen   
2         2.89   0.869565    0.235640      3  20230407  C0407bb  PD_Screen   
3         2.72   0.906977    0.218015      4  20230407  C0407be  PD_Screen   
4         2.64   0.945946    0.279924      1  20220524  B0520ba  PD_Screen   
..         ...        ...         ...    ...       ...      ...        ...   
640       3.75   0.605263    0.179467      7  20221014  C0928ca  PD_Screen   
641       3.04   0.645161    0.215132      8  20221014  A1014cb  PD_Screen   
642       3.99   0.708333    0.149373      9  20221014  B1014cc  PD_Screen   
643       3.62   0.704545    0.150829     10  20221014  C1014cd  PD_Screen   
644       2.97   0.724138    0.197980     11  20221014  A1014ce  PD_Screen   

            dataset      Gene  Allele  ...  habit_dura  habit_p

### The below cell reads in file from online repository (github) - example in case this is possible in the future

In [ ]:
# # tap_url = 'https://osf.io/du9bj/files/osfstorage/650a2f9f1e76a4230e8a99a5?raw=true'
# tap_url='https://github.com/MyYummyPancake/NRSC510B/blob/main/tap_output.csv?raw=true'
# # s=requests.get(tap_url).content
# # tap_output=pd.read_csv(io.StringIO(s.decode('utf-8')))
# tap_output=pd.read_csv(tap_url, on_bad_lines='skip', index_col=0)
# print(tap_output)

In [ ]:
# print(tap_output['Gene'].unique())
# print(len(tap_output['Gene'].unique()))
# print(baseline_output['Gene'].unique())
# print(len(baseline_output['Gene'].unique()))

In [ ]:
# for x in tap_output['Gene'].unique():
#     tap_output_gene=tap_output[tap_output['Gene']==x]
#     gene_tap_data=tap_output[tap_output['Date'].isin(tap_output_gene['Date'].unique())]
#     gene_tap_data_final=gene_tap_data[gene_tap_data['Gene'].isin(['N2', x])]
#     gene_tap_data_final['taps']=gene_tap_data_final['taps'].astype(int)


# for x in baseline_output['Gene'].unique():
#     baseline_output_gene=baseline_output[baseline_output['Gene']==x]
#     gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
#     gene_baseline_data_final=gene_baseline_data[gene_baseline_data['Gene'].isin(['N2', x])]
#     for a,b in zip(list_baseline_metrics, list_baseline_Tstats):
#         TTest_Allele(x, a, baseline_output, b)


# Run Statistics (T-Test and sample-mean distance) on Data

In [11]:
import pingouin as pg
from pingouin import ttest
from scipy.stats import ttest_ind
import tqdm

In [12]:
print(baseline_output)

            Time    n  Number  Instantaneous Speed  Interval Speed   Bias  \
0        490.055   56      39               0.0471          0.0548  0.256   
1        490.135   56      39               0.0507          0.0565  0.256   
2        490.212   56      39               0.0503          0.0546  0.256   
3        490.292   56      39               0.0450          0.0481  0.256   
4        490.378   56      39               0.0439          0.0455  0.179   
...          ...  ...     ...                  ...             ...    ...   
1104538  589.610  113      82               0.0830          0.0923  0.241   
1104539  589.684  113      82               0.0790          0.0862  0.215   
1104540  589.771  113      82               0.0748          0.0825  0.228   
1104541  589.851  113      82               0.0757          0.0788  0.291   
1104542  589.932  112      80               0.0734          0.0761  0.338   

          Width  Length      Area  Angular Speed  Aspect Ratio  Kink  Curve

## Sample Mean Distance & Confidence Interval Calculation of Baseline Data
1. Calculate Mean

2. Calculate SEM

3. Calculate CI

4. Calculate Mean Distance From Control

In [13]:
print(tap_data.columns)

Index(['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed', 'recov_dura',
       'dist', 'recov_prob', 'recov_speed', 'recovery_dura', 'recovery_prob',
       'recovery_speed'],
      dtype='object')


In [80]:
baseline_output_byplate=baseline_output.groupby(by=['Plate_id',
                                                    'Date',
                                                    'Screen',
                                                    'dataset',
                                                    'Gene',
                                                    'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                                    'n',
                                                                                                    'Number',
                                                                                                    'Time',
                                                                                                    'Screen',
                                                                                                    'dataset',
                                                                                                    'Allele'])
print(baseline_output_byplate)
print('-'*30)
print(baseline_output_byplate.columns[1:])
# print(baseline_output_byplate[['Gene','Instantaneous Speed']].copy())

print('-'*30)
tap_data_byplate=tap_data.groupby(by=['Plate_id',
                                    'Date',
                                    'Screen',
                                    'dataset',
                                    'Gene',
                                    'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                    'Screen',
                                                                                    'dataset',
                                                                                    'Allele',
                                                                                    'dist',
                                                                                    'plate',
                                                                                    'recov_dura',
                                                                                    'recov_prob',
                                                                                    'recov_speed'])


print(tap_data_byplate)
print('-'*30)
print(tap_data_byplate.columns)
print('-'*50)

print("allele output")
print('-'*50)
baseline_output_allele_byplate=baseline_output.groupby(by=['Plate_id',
                                                            'Date',
                                                            'Screen',
                                                            'dataset',
                                                            'Gene',
                                                            'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                                            'n',
                                                                                                            'Number',
                                                                                                            'Time',
                                                                                                            'Screen',
                                                                                                           'Date',
                                                                                                            'Gene',
                                                                                                            'Allele'])
print(baseline_output_allele_byplate)
print('-'*50)
print(baseline_output_allele_byplate.columns)
# print(baseline_output_byplate[['Gene','Instantaneous Speed']].copy())

print('-'*50)
tap_data_allele_byplate=tap_data.groupby(by=['Plate_id',
                                            'Date',
                                            'Screen',
                                            'dataset',
                                            'Gene',
                                            'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                            'Screen',
                                                                                            'Gene',
                                                                                           'Date',
                                                                                            'Allele',
                                                                                            'dist',
                                                                                            'plate',
                                                                                            'recov_dura',
                                                                                            'recov_prob',
                                                                                            'recov_speed'])


print(tap_data_allele_byplate)
print('-'*50)
print(tap_data_allele_byplate.columns)

         Date   Gene  Instantaneous Speed  Interval Speed      Bias     Width  \
0    20230125     N2             0.053020        0.054984  0.109245  0.121230   
1    20230125     N2             0.034548        0.038413  0.022895  0.136490   
2    20230125   pamn             0.054918        0.064569  0.038388  0.100071   
3    20230125   pamn             0.053658        0.062931  0.057211  0.097976   
4    20230127     N2             0.040873        0.042333  0.078969  0.108768   
..        ...    ...                  ...             ...       ...       ...   
651  20221206     N2             0.044065        0.053004  0.110147  0.136670   
652  20221206     N2             0.044000        0.054308  0.144534  0.128048   
653  20221206  gba-4             0.056177        0.059488  0.215189  0.119054   
654  20221206  gba-4             0.039395        0.041627  0.143595  0.112501   
655  20221206  gba-4             0.044262        0.048581  0.140027  0.110496   

       Length      Area  An

In [78]:
''' 
input: one pd.DataFrame with multiple columns (1st column is the index, the other are phenotypes)
return: list of pd.DataFrame, each DataFrame with 2 columns: index and 1 phenotype, 
        for how many phenotypes there are in the input
'''
def extract_phenotypes(df):
    list_phenotypes_df = []
    phenotypes = df.columns[1:]
    index = df.columns[0]
    for i in phenotypes:
        new_df = df[[index, i]].copy()
        list_phenotypes_df.append(new_df)
    return list_phenotypes_df



'''
input: list of dataframes with 2 columns (1st one is index, 2nd one are indiviudal phenotypes)
        output from extract_phenotypes function

output: the same list of dataframes with additional calculated columns for:
                1. mean
                2. count
                3. std
                4. ci95_hi
                5. ci95_low
                6. ci95
'''
        # baseline_output_gene=baseline_output[baseline_output['dataset']==x]
        # gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        # gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]

def calculate_MSD(list_of_dfs):
    new_list_of_dfs=[]
    for i in list_of_dfs:
        calculate_1= i.groupby(by=['Gene'], as_index=False).agg(['mean', 'count', 'sem'])
        calculate_2= ci95(calculate_1)
        N2_data = calculate_2[calculate_2.index=='N2']
        calculate_2.iloc[:,0] = calculate_2.iloc[:,0]-N2_data.iloc[0,0]
        calculate_2.iloc[:,3] = calculate_2.iloc[:,3]-N2_data.iloc[0,0]
        calculate_2.iloc[:,4] = calculate_2.iloc[:,4]-N2_data.iloc[0,0]
      
        
        # calculate_2.columns=['_'.join(col) for col in calculate_2.columns.values]
        new_list_of_dfs.append(calculate_2)
    return new_list_of_dfs
    


'''
input: df of 4 columns: index, mean, count, std

output: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

'''
def ci95(df):
    for metric in df.columns.levels[0]:
        if metric == 'Gene':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m, c, s = df[metric].loc[i]
                ci95_hi.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
    return df



'''
input: list of dataframes with 2 columns (1st one is index, 2nd one are indiviudal phenotypes)
        output from extract_phenotypes function

output: the same list of dataframes with additional calculated columns for:
                1. mean
                2. count
                3. std
                4. ci95_hi
                5. ci95_low
                6. ci95
'''
        # baseline_output_gene=baseline_output[baseline_output['dataset']==x]
        # gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        # gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]

def calculate_allele_MSD(list_of_dfs):
    new_list_of_dfs=[]
    for i in list_of_dfs:
        calculate_1= i.groupby(by=['dataset'], as_index=False).agg(['mean', 'count', 'sem'])
        calculate_2= ci95_allele(calculate_1)
        N2_data = calculate_2[calculate_2.index=='N2']
        calculate_2.iloc[:,0] = calculate_2.iloc[:,0]-N2_data.iloc[0,0]
        calculate_2.iloc[:,3] = calculate_2.iloc[:,3]-N2_data.iloc[0,0]
        calculate_2.iloc[:,4] = calculate_2.iloc[:,4]-N2_data.iloc[0,0]
        # calculate_2.columns=['_'.join(col) for col in calculate_2.columns.values]
        new_list_of_dfs.append(calculate_2)
    return new_list_of_dfs
    
        
'''
input: df of 4 columns: index, mean, count, std

output: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

'''
def ci95_allele(df):
    for metric in df.columns.levels[0]:
        if metric == 'dataset':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m, c, s = df[metric].loc[i]
                ci95_hi.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
    return df



In [54]:
list_baseline_MSD=calculate_MSD(extract_phenotypes(baseline_output_byplate))
print(list_baseline_MSD)

list_tap_MSD=calculate_MSD(extract_phenotypes(tap_data_byplate))
print(list_tap_MSD)

[           Instantaneous Speed                                    
                          mean count       sem   ci95_hi   ci95_lo
Gene                                                              
F32D8.13             -0.011842     4  0.007432  0.011811 -0.035494
F53B2.5               0.089373     5  0.007332  0.109730  0.069016
N2                    0.000000   161  0.001208  0.002385 -0.002385
T20F7.3              -0.011105     9  0.001642 -0.007319 -0.014891
Y18D10A.22            0.006568     5  0.006317  0.024108 -0.010972
bas-1                 0.139027    10  0.019468  0.183066  0.094989
cat-1                 0.053776     9  0.014488  0.087184  0.020368
cat-2                 0.058272    15  0.006819  0.072898  0.043647
clh-5                -0.013683     4  0.003259 -0.003313 -0.024054
cpr-1                 0.007414     5  0.012109  0.041032 -0.026205
dat-1                 0.032528     5  0.007951  0.054604  0.010452
djr-1.2              -0.009114     5  0.002430 -0.002367 -0.0

/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/3987244237.py:60: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_hi.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[1])
/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/3987244237.py:61: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_lo.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[0])
/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/3987244237.py:60: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_hi.appen

In [56]:
for a in list_baseline_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_baseline_MSD[0].columns.levels[0]:
        baseline_MSD=a
    else:
        baseline_MSD=baseline_MSD.join(a)

for a in list_tap_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_tap_MSD[0].columns.levels[0]:
        tap_MSD=a
    else:
        tap_MSD=tap_MSD.join(a)

In [57]:
print(baseline_MSD.columns.levels[0])
print("-"*30)
print(tap_MSD.columns.levels[0])

Index(['Angular Speed', 'Area', 'Aspect Ratio', 'Bias', 'Crab', 'Curve',
       'Instantaneous Speed', 'Interval Speed', 'Kink', 'Length', 'Width'],
      dtype='object')
------------------------------
Index(['final_dura', 'final_prob', 'final_speed', 'habit_dura', 'habit_prob',
       'habit_speed', 'init_dura', 'init_prob', 'init_speed', 'recovery_dura',
       'recovery_prob', 'recovery_speed'],
      dtype='object')


In [58]:
test_df=list_baseline_MSD[-1]
# test_df['Crab']
test_df.columns.levels[0]

Index(['Crab'], dtype='object')

In [59]:
combined_MSD = baseline_MSD.join(tap_MSD, on='Gene')
print(combined_MSD.columns.levels[0])


Index(['Angular Speed', 'Area', 'Aspect Ratio', 'Bias', 'Crab', 'Curve',
       'Instantaneous Speed', 'Interval Speed', 'Kink', 'Length', 'Width',
       'final_dura', 'final_prob', 'final_speed', 'habit_dura', 'habit_prob',
       'habit_speed', 'init_dura', 'init_prob', 'init_speed', 'recovery_dura',
       'recovery_prob', 'recovery_speed'],
      dtype='object')


In [60]:
combined_MSD=combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                         "habit_prob": "Habituation of Respones Probability",
                                         "habit_speed":"Habituation of Response Speed",
                                         "init_dura": "Initial Response Duration",
                                         "init_prob": "Initial Response Probability",
                                         "init_speed": "Initial Response Speed",
                                         "final_dura": "Final Response Duration",
                                         "final_prob": "Final Response Probability",
                                         "final_speed": "Final Response Speed",
                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                         "recovery_speed": "Spontaneous Recovery of Response Speed"})
print(combined_MSD.columns.levels[0])
print(len(combined_MSD.columns.levels[0]))

Index(['Angular Speed', 'Area', 'Aspect Ratio', 'Bias', 'Crab', 'Curve',
       'Final Response Duration', 'Final Response Probability',
       'Final Response Speed', 'Habituation of Respones Probability',
       'Habituation of Response Duration', 'Habituation of Response Speed',
       'Initial Response Duration', 'Initial Response Probability',
       'Initial Response Speed', 'Instantaneous Speed', 'Interval Speed',
       'Kink', 'Length', 'Spontaneous Recovery of Response Duration',
       'Spontaneous Recovery of Response Probability',
       'Spontaneous Recovery of Response Speed', 'Width'],
      dtype='object')
23


In [61]:
combined_MSD=combined_MSD.reset_index()
combined_MSD.columns = combined_MSD.columns.to_flat_index().str.join('-')
print(combined_MSD)

         Gene-  Instantaneous Speed-mean  Instantaneous Speed-count  \
0     F32D8.13                 -0.011842                          4   
1      F53B2.5                  0.089373                          5   
2           N2                  0.000000                        161   
3      T20F7.3                 -0.011105                          9   
4   Y18D10A.22                  0.006568                          5   
5        bas-1                  0.139027                         10   
6        cat-1                  0.053776                          9   
7        cat-2                  0.058272                         15   
8        clh-5                 -0.013683                          4   
9        cpr-1                  0.007414                          5   
10       dat-1                  0.032528                          5   
11     djr-1.2                 -0.009114                          5   
12      dnj-25                 -0.011163                          5   
13    

## Allele-level SMD

In [85]:
list_allele_baseline_MSD=calculate_allele_MSD(extract_phenotypes(baseline_output_allele_byplate))
print(list_allele_baseline_MSD)

list_allele_tap_MSD=calculate_allele_MSD(extract_phenotypes(tap_data_allele_byplate))
print(list_allele_tap_MSD)

/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/946989816.py:119: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_hi.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[1])
/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/946989816.py:120: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_lo.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[0])
/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/946989816.py:119: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_hi.appen

[                  Instantaneous Speed                                    
                                 mean count       sem   ci95_hi   ci95_lo
dataset                                                                  
F32D8.13_gk584              -0.011842     4  0.007432  0.011811 -0.035494
F53B2.5_ok226                0.089373     5  0.007332  0.109730  0.069016
N2                           0.000000   161  0.001208  0.002385 -0.002385
T20F7.3_tm2936              -0.011105     9  0.001642 -0.007319 -0.014891
Y18D10A.22_ok3535            0.006568     5  0.006317  0.024108 -0.010972
...                               ...   ...       ...       ...       ...
unc-57_e406                 -0.013206     4  0.003900 -0.000795 -0.025617
unc-57_ok310                -0.015077     5  0.003245 -0.006066 -0.024087
unc-59_e1005                -0.010957     5  0.004158  0.000587 -0.022502
unc-59_e261                 -0.009768     5  0.002405 -0.003091 -0.016445
vps-35_ok1880               -0.005118

/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/946989816.py:119: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_hi.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[1])
/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/946989816.py:120: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_lo.append(stats.t.interval(alpha=0.95, df=c-1, loc=m, scale=s)[0])
/var/folders/31/jpj6mpsx60d9829cymplqfsh0000gn/T/ipykernel_24838/946989816.py:119: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  ci95_hi.appen

In [86]:
for a in list_allele_baseline_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_allele_baseline_MSD[0].columns.levels[0]:
        allele_baseline_MSD=a
    else:
        allele_baseline_MSD=allele_baseline_MSD.join(a)

for a in list_allele_tap_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_allele_tap_MSD[0].columns.levels[0]:
        allele_tap_MSD=a
    else:
        allele_tap_MSD=allele_tap_MSD.join(a)

In [89]:
print(allele_baseline_MSD.columns.levels[0])
print("-"*30)
print(allele_tap_MSD.columns.levels[0])

Index(['Angular Speed', 'Area', 'Aspect Ratio', 'Bias', 'Crab', 'Curve',
       'Instantaneous Speed', 'Interval Speed', 'Kink', 'Length', 'Width'],
      dtype='object')
------------------------------
Index(['final_dura', 'final_prob', 'final_speed', 'habit_dura', 'habit_prob',
       'habit_speed', 'init_dura', 'init_prob', 'init_speed', 'recovery_dura',
       'recovery_prob', 'recovery_speed'],
      dtype='object')


In [90]:
allele_combined_MSD = allele_baseline_MSD.join(allele_tap_MSD, on='dataset')
print(allele_combined_MSD.columns.levels[0])


Index(['Angular Speed', 'Area', 'Aspect Ratio', 'Bias', 'Crab', 'Curve',
       'Instantaneous Speed', 'Interval Speed', 'Kink', 'Length', 'Width',
       'final_dura', 'final_prob', 'final_speed', 'habit_dura', 'habit_prob',
       'habit_speed', 'init_dura', 'init_prob', 'init_speed', 'recovery_dura',
       'recovery_prob', 'recovery_speed'],
      dtype='object')


In [91]:
allele_combined_MSD=allele_combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                                         "habit_prob": "Habituation of Respones Probability",
                                                         "habit_speed":"Habituation of Response Speed",
                                                         "init_dura": "Initial Response Duration",
                                                         "init_prob": "Initial Response Probability",
                                                         "init_speed": "Initial Response Speed",
                                                         "final_dura": "Final Response Duration",
                                                         "final_prob": "Final Response Probability",
                                                         "final_speed": "Final Response Speed",
                                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                                         "recovery_speed": "Spontaneous Recovery of Response Speed"})
print(allele_combined_MSD.columns.levels[0])
print(len(allele_combined_MSD.columns.levels[0]))

Index(['Angular Speed', 'Area', 'Aspect Ratio', 'Bias', 'Crab', 'Curve',
       'Final Response Duration', 'Final Response Probability',
       'Final Response Speed', 'Habituation of Respones Probability',
       'Habituation of Response Duration', 'Habituation of Response Speed',
       'Initial Response Duration', 'Initial Response Probability',
       'Initial Response Speed', 'Instantaneous Speed', 'Interval Speed',
       'Kink', 'Length', 'Spontaneous Recovery of Response Duration',
       'Spontaneous Recovery of Response Probability',
       'Spontaneous Recovery of Response Speed', 'Width'],
      dtype='object')
23


In [92]:
allele_combined_MSD=allele_combined_MSD.reset_index()
allele_combined_MSD.columns = allele_combined_MSD.columns.to_flat_index().str.join('-')
print(allele_combined_MSD)

             dataset-  Instantaneous Speed-mean  Instantaneous Speed-count  \
0      F32D8.13_gk584                 -0.011842                          4   
1       F53B2.5_ok226                  0.089373                          5   
2                  N2                  0.000000                        161   
3      T20F7.3_tm2936                 -0.011105                          9   
4   Y18D10A.22_ok3535                  0.006568                          5   
..                ...                       ...                        ...   
86        unc-57_e406                 -0.013206                          4   
87       unc-57_ok310                 -0.015077                          5   
88       unc-59_e1005                 -0.010957                          5   
89        unc-59_e261                 -0.009768                          5   
90      vps-35_ok1880                 -0.005118                         11   

    Instantaneous Speed-sem  Instantaneous Speed-ci95_hi  \
0  

### Allele-level T-stat analysis of baseline data

In [ ]:
list=baseline_output["dataset"].unique()
print(list)
print(len(list))
print(baseline_output)

In [ ]:
def TTest_Allele(Type, DF_ref, output):
    for a in DF_ref['dataset'].unique():
        Tstat=ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele == "N2"][Type],equal_var=False)[0]
        row=[a, Tstat]
        output.loc[len(output)]=row
    print(output)

In [ ]:
PD_baseline_instantspeed_T_allele=pd.DataFrame(columns = ["dataset","Instantaneous Speed"])
PD_baseline_intspeed_T_allele=pd.DataFrame(columns = ["dataset","Interval Speed"])
PD_baseline_bias_T_allele=pd.DataFrame(columns = ["dataset","Bias"])
PD_baseline_width_T_allele=pd.DataFrame(columns = ["dataset","Width"])
PD_baseline_length_T_allele=pd.DataFrame(columns = ["dataset","Length"])
PD_baseline_area_T_allele=pd.DataFrame(columns = ["dataset","Area"])
PD_baseline_angularspeed_T_allele=pd.DataFrame(columns = ["dataset","Angular Speed"])
PD_baseline_aspectratio_T_allele=pd.DataFrame(columns = ["dataset","Aspect Ratio"])
PD_baseline_kink_T_allele=pd.DataFrame(columns = ["dataset","Kink"])
PD_baseline_curve_T_allele=pd.DataFrame(columns = ["dataset","Curve"])
PD_baseline_crab_T_allele=pd.DataFrame(columns = ["dataset","Crab"])

list_baseline_Tstats_allele=[PD_baseline_instantspeed_T_allele,
                      PD_baseline_intspeed_T_allele,
                      PD_baseline_bias_T_allele,
                      PD_baseline_width_T_allele,
                      PD_baseline_length_T_allele,
                      PD_baseline_area_T_allele,
                      PD_baseline_angularspeed_T_allele,
                      PD_baseline_aspectratio_T_allele,
                      PD_baseline_kink_T_allele,
                      PD_baseline_curve_T_allele,
                      PD_baseline_crab_T_allele]

list_baseline_metrics=["Instantaneous Speed",
                       "Interval Speed",
                       "Bias",
                       "Width",
                       "Length",
                       "Area",
                       "Angular Speed",
                       "Aspect Ratio",
                       "Kink",
                       "Curve",
                       "Crab"]


In [ ]:
for x in baseline_output['dataset'].unique():
    if x == "N2":
        pass
    else:
        baseline_output_gene=baseline_output[baseline_output['dataset']==x]
        gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_baseline_metrics, list_baseline_Tstats_allele):
            TTest_Allele(a, gene_baseline_data_final, b)

In [ ]:
PD_baseline_Tstats_allele=pd.DataFrame()
for a in list_baseline_Tstats_allele:
    b=a.groupby(['dataset'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_baseline_Tstats_allele[0].columns.values[1]:
        PD_baseline_Tstats_allele=b
    else:
        PD_baseline_Tstats_allele=PD_baseline_Tstats_allele.join(b.iloc[:,1])
        
print(PD_baseline_Tstats_allele)

In [ ]:
PD_baseline_Tstats_allele=PD_baseline_Tstats_allele.set_index("dataset")
print(PD_baseline_Tstats_allele)
PD_baseline_Tstats_allele_sort=PD_baseline_Tstats_allele.sort_index()
print(PD_baseline_Tstats_allele_sort)

### Gene-level T-stat analysis of baseline data

In [ ]:
def TTest_Gene(Type, DF_ref, output):
    for a in DF_ref['Gene'].unique():
        Tstat=ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        row=[a, Tstat]
        output.loc[len(output)]=row
    print(output)

In [ ]:
PD_baseline_instantspeed_T=pd.DataFrame(columns = ["Gene","Instantaneous Speed"])
PD_baseline_intspeed_T=pd.DataFrame(columns = ["Gene","Interval Speed"])
PD_baseline_bias_T=pd.DataFrame(columns = ["Gene","Bias"])
PD_baseline_width_T=pd.DataFrame(columns = ["Gene","Width"])
PD_baseline_length_T=pd.DataFrame(columns = ["Gene","Length"])
PD_baseline_area_T=pd.DataFrame(columns = ["Gene","Area"])
PD_baseline_angularspeed_T=pd.DataFrame(columns = ["Gene","Angular Speed"])
PD_baseline_aspectratio_T=pd.DataFrame(columns = ["Gene","Aspect Ratio"])
PD_baseline_kink_T=pd.DataFrame(columns = ["Gene","Kink"])
PD_baseline_curve_T=pd.DataFrame(columns = ["Gene","Curve"])
PD_baseline_crab_T=pd.DataFrame(columns = ["Gene","Crab"])

list_baseline_Tstats=[PD_baseline_instantspeed_T,
                      PD_baseline_intspeed_T,
                      PD_baseline_bias_T,
                      PD_baseline_width_T,
                      PD_baseline_length_T,
                      PD_baseline_area_T,
                      PD_baseline_angularspeed_T,
                      PD_baseline_aspectratio_T,
                      PD_baseline_kink_T,
                      PD_baseline_curve_T,
                      PD_baseline_crab_T]

list_baseline_metrics=["Instantaneous Speed",
                       "Interval Speed",
                       "Bias",
                       "Width",
                       "Length",
                       "Area",
                       "Angular Speed",
                       "Aspect Ratio",
                       "Kink",
                       "Curve",
                       "Crab"]

In [ ]:
for x in baseline_output['Gene'].unique():
    if x == "N2":
        pass
    else:
        baseline_output_gene=baseline_output[baseline_output['Gene']==x]
        gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        gene_baseline_data_final=gene_baseline_data[gene_baseline_data['Gene'].isin(['N2', x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_baseline_metrics, list_baseline_Tstats):
            TTest_Gene(a, gene_baseline_data_final, b)
            

In [ ]:
PD_baseline_Tstats=pd.DataFrame()
for a in list_baseline_Tstats:
    b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_baseline_Tstats[0].columns.values[1]:
        PD_baseline_Tstats=b
    else:
        PD_baseline_Tstats=PD_baseline_Tstats.join(b.iloc[:,1])
        
print(PD_baseline_Tstats)

In [ ]:
PD_baseline_Tstats=PD_baseline_Tstats.set_index("Gene")
print(PD_baseline_Tstats)
PD_baseline_Tstats_sort=PD_baseline_Tstats.sort_index()
print(PD_baseline_Tstats_sort)

### T-stat analysis for tap-response data

In [ ]:
# PD_tap=tap_output
# PD_first_tap = PD_tap[(PD_tap.taps==1)].reset_index().drop(columns="index")
# PD_last_three_taps = PD_tap[((PD_tap.taps>=28)&(PD_tap.taps<=30))].reset_index().drop(columns="index")
# PD_recov_taps = PD_tap[(PD_tap.taps==31)].reset_index().drop(columns="index")
# print(PD_first_tap)
# print(PD_last_three_taps)
# print(PD_recov_taps)
# PD_final_taps=PD_last_three_taps.groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"]).mean().reset_index()
# print(PD_final_taps)

In [ ]:
# PD_final_taps=PD_final_taps.rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")
# PD_first_tap=PD_first_tap.rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
# print(PD_final_taps)
# print(PD_first_tap)

In [ ]:
# PD_habit_levels = pd.merge(PD_first_tap, PD_final_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
# PD_habit_levels = PD_habit_levels.dropna()
# print(PD_habit_levels)

In [ ]:
# PD_habit_levels['habit_dura']=PD_habit_levels.init_dura-PD_habit_levels.final_dura
# PD_habit_levels['habit_prob']=PD_habit_levels.init_prob-PD_habit_levels.final_prob
# PD_habit_levels['habit_speed']=PD_habit_levels.init_speed-PD_habit_levels.final_speed
# # PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y', 'Allele','Gene'])
# PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y'])

In [ ]:
# PD_recov_taps=PD_recov_taps.rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
# PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
# PD_habituation = PD_habituation.drop(columns=['taps_x','taps_y','taps','time'])
# PD_habituation = PD_habituation.dropna()
# print(PD_habituation)

In [ ]:
# PD_habituation['recovery_dura']=PD_habituation.init_dura-PD_habituation.recov_dura
# PD_habituation['recovery_prob']=PD_habituation.init_prob-PD_habituation.recov_prob
# PD_habituation['recovery_speed']=PD_habituation.init_speed-PD_habituation.recov_speed
# PD_habituation=PD_habituation.dropna()
# print(PD_habituation)

In [ ]:
print(tap_data)

In [ ]:
print(tap_data.columns.values)

## Allele level T-stat analysis of tap response data

In [ ]:
recovery_dura_allele=pd.DataFrame(columns = ["dataset","Recovery Duration"])
recovery_prob_allele=pd.DataFrame(columns = ["dataset","Recovery Probability"])
recovery_speed_allele=pd.DataFrame(columns = ["dataset","Recovery Speed"])
init_dura_allele=pd.DataFrame(columns = ["dataset","Initial Duration"])
init_prob_allele=pd.DataFrame(columns = ["dataset","Initial Probability"])
init_speed_allele=pd.DataFrame(columns = ["dataset","Initial Speed"])
final_dura_allele=pd.DataFrame(columns = ["dataset","Final Duration"])
final_prob_allele=pd.DataFrame(columns = ["dataset","Final Probability"])
final_speed_allele=pd.DataFrame(columns = ["dataset","Final Speed"])
hab_dura_allele=pd.DataFrame(columns = ["dataset","Habituation of Duration"])
hab_prob_allele=pd.DataFrame(columns = ["dataset","Habituation of Probability"])
hab_speed_allele=pd.DataFrame(columns = ["dataset","Habituation of Speed"])

list_tap_Tstats_allele = [recovery_dura_allele,
                   recovery_prob_allele,
                   recovery_speed_allele,
                   init_dura_allele,
                   init_prob_allele,
                   init_speed_allele,
                   final_dura_allele,
                   final_prob_allele,
                   final_speed_allele,
                   hab_dura_allele,
                   hab_prob_allele,
                   hab_speed_allele]
list_tap_metrics = ["recovery_dura",
                    "recovery_prob",
                    "recovery_speed",
                    "init_dura",
                    "init_prob",
                    "init_speed",
                    "final_dura",
                    "final_prob",
                    "final_speed",
                    "habit_dura",
                    "habit_prob",
                    "habit_speed"]
print('done')

In [ ]:
for x in tap_data['dataset'].unique():
    if x == "N2":
        pass
    else:
        PD_habituation_allele=tap_data[tap_data['dataset']==x]
        gene_PD_habituation_data=tap_data[tap_data['Date'].isin(PD_habituation_allele['Date'].unique())]
        gene_PD_habituation_data_final=gene_PD_habituation_data[gene_PD_habituation_data['dataset'].isin(['N2', x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_tap_metrics, list_tap_Tstats_allele):
            TTest_Allele(a, gene_PD_habituation_data_final, b)

In [ ]:
PD_habituation_Tstats_allele=pd.DataFrame()
for a in list_tap_Tstats_allele:
    b=a.groupby(['dataset'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_tap_Tstats_allele[0].columns.values[1]:
        PD_habituation_Tstats_allele=b
    else:
        PD_habituation_Tstats_allele=PD_habituation_Tstats_allele.join(b.iloc[:,1])
        
print(PD_habituation_Tstats_allele)

In [ ]:
PD_habituation_Tstats_allele=PD_habituation_Tstats_allele.set_index("dataset")
print(PD_habituation_Tstats_allele)
PD_habituation_Tstats_allele_sorted=PD_habituation_Tstats_allele.sort_index()
print(PD_habituation_Tstats_allele_sorted)
print(PD_baseline_Tstats_allele_sort)

## Gene-level T-stat analysis of Tap response data

In [ ]:
recovery_dura=pd.DataFrame(columns = ["Gene","Recovery Duration"])
recovery_prob=pd.DataFrame(columns = ["Gene","Recovery Probability"])
recovery_speed=pd.DataFrame(columns = ["Gene","Recovery Speed"])
init_dura=pd.DataFrame(columns = ["Gene","Initial Duration"])
init_prob=pd.DataFrame(columns = ["Gene","Initial Probability"])
init_speed=pd.DataFrame(columns = ["Gene","Initial Speed"])
final_dura=pd.DataFrame(columns = ["Gene","Final Duration"])
final_prob=pd.DataFrame(columns = ["Gene","Final Probability"])
final_speed=pd.DataFrame(columns = ["Gene","Final Speed"])
hab_dura=pd.DataFrame(columns = ["Gene","Habituation of Duration"])
hab_prob=pd.DataFrame(columns = ["Gene","Habituation of Probability"])
hab_speed=pd.DataFrame(columns = ["Gene","Habituation of Speed"])

list_tap_Tstats = [recovery_dura,
                   recovery_prob,
                   recovery_speed,
                   init_dura,
                   init_prob,
                   init_speed,
                   final_dura,
                   final_prob,
                   final_speed,
                   hab_dura,
                   hab_prob,
                   hab_speed]
list_tap_metrics = ["recovery_dura",
                    "recovery_prob",
                    "recovery_speed",
                    "init_dura",
                    "init_prob",
                    "init_speed",
                    "final_dura",
                    "final_prob",
                    "final_speed",
                    "habit_dura",
                    "habit_prob",
                    "habit_speed"]
print('done')

In [ ]:
for x in tap_data['Gene'].unique():
    if x == "N2":
        pass
    else:
        PD_habituation_gene=tap_data[tap_data['Gene']==x]
        gene_PD_habituation_data=tap_data[tap_data['Date'].isin(PD_habituation_gene['Date'].unique())]
        gene_PD_habituation_data_final=gene_PD_habituation_data[gene_PD_habituation_data['Gene'].isin(['N2', x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_tap_metrics, list_tap_Tstats):
            TTest_Gene(a, gene_PD_habituation_data_final, b)

In [ ]:
PD_habituation_Tstats=pd.DataFrame()
for a in list_tap_Tstats:
    b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_tap_Tstats[0].columns.values[1]:
        PD_habituation_Tstats=b
    else:
        PD_habituation_Tstats=PD_habituation_Tstats.join(b.iloc[:,1])
        
print(PD_habituation_Tstats)

In [ ]:
PD_habituation_Tstats=PD_habituation_Tstats.set_index("Gene")
PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()
print(PD_habituation_Tstats)
print(PD_habituation_Tstats_sorted)
print(PD_baseline_Tstats_sort)

# Merging t-stat data into one dataset

## Gene-level

In [ ]:
combined_Tstats = pd.merge(PD_baseline_Tstats, PD_habituation_Tstats, on =['Gene'], how ='left')
combined_Tstats=combined_Tstats.sort_index()
# PD_combined_Tstats=PD_combined_Tstats.drop(index="N2")
combined_Tstats=combined_Tstats.dropna()
print(combined_Tstats)
combined_Tstats_normalize=(combined_Tstats-combined_Tstats.mean())/combined_Tstats.std()
print(combined_Tstats_normalize)
print(combined_Tstats_normalize[combined_Tstats_normalize.index=="N2"])
print(combined_Tstats_normalize.index)
combined_Tstats_normalize_2=combined_Tstats_normalize-combined_Tstats_normalize[combined_Tstats_normalize.index=="N2"].squeeze()
print(combined_Tstats_normalize_2[combined_Tstats_normalize_2.index=="N2"])

first_col=combined_Tstats.pop("Area")
combined_Tstats.insert(0,"Area",first_col)

first_col=combined_Tstats.pop("Length")
combined_Tstats.insert(0,"Length",first_col)

first_col=combined_Tstats.pop("Width")
combined_Tstats.insert(0,"Width",first_col)

first_col=combined_Tstats.pop("Angular Speed")
combined_Tstats.insert(5,"Angular Speed",first_col)

print(combined_Tstats)

combined_Tstats=combined_Tstats.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                 "Habituation of Probability": "Habituation of Respones Probability",
                                                 "Habituation of Speed":"Habituation of Response Speed",
                                                 "Initial Duration": "Initial Response Duration",
                                                 "Initial Probability": "Initial Response Probability",
                                                 "Initial Speed": "Initial Response Speed",
                                                 "Final Duration": "Final Response Duration",
                                                 "Final Probability": "Final Response Probability",
                                                 "Final Speed": "Final Response Speed",
                                                 "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                 "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                 "Recovery Speed": "Spontaneous Recovery of Response Speed"})

combined_Tstats_normalize_2=combined_Tstats_normalize_2.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                                         "Habituation of Probability": "Habituation of Respones Probability",
                                                                         "Habituation of Speed":"Habituation of Response Speed",
                                                                         "Initial Duration": "Initial Response Duration",
                                                                         "Initial Probability": "Initial Response Probability",
                                                                         "Initial Speed": "Initial Response Speed",
                                                                         "Final Duration": "Final Response Duration",
                                                                         "Final Probability": "Final Response Probability",
                                                                         "Final Speed": "Final Response Speed",
                                                                         "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                                         "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                                         "Recovery Speed": "Spontaneous Recovery of Response Speed"})
print(combined_Tstats_normalize_2)

first_col=combined_Tstats_normalize_2.pop("Area")
combined_Tstats_normalize_2.insert(0,"Area",first_col)

first_col=combined_Tstats_normalize_2.pop("Length")
combined_Tstats_normalize_2.insert(0,"Length",first_col)

first_col=combined_Tstats_normalize_2.pop("Width")
combined_Tstats_normalize_2.insert(0,"Width",first_col)

first_col=combined_Tstats_normalize_2.pop("Angular Speed")
combined_Tstats_normalize_2.insert(5,"Angular Speed",first_col)

last_col=combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Duration")
combined_Tstats_normalize_2.insert(22,"Spontaneous Recovery of Response Duration",last_col)

last_col=combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Probability")
combined_Tstats_normalize_2.insert(22,"Spontaneous Recovery of Response Probability",last_col)

last_col=combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Speed")
combined_Tstats_normalize_2.insert(22,"Spontaneous Recovery of Response Speed",last_col)

print(combined_Tstats_normalize_2)

combined_Tstats_melted=combined_Tstats.reset_index()
combined_Tstats_melted=pd.melt(combined_Tstats_melted, id_vars=['Gene'],
                               var_name='Metric',
                               value_name='T_score')
combined_Tstats_melted_sorted=combined_Tstats_melted.sort_values(by=['T_score'])
print(combined_Tstats_melted_sorted)

combined_Tstats_normalized_melted=combined_Tstats_normalize_2.reset_index()
combined_Tstats_normalized_melted=pd.melt(combined_Tstats_normalized_melted, id_vars=['Gene'],
                                           var_name='Metric',
                                           value_name='T_score')
print(combined_Tstats_normalized_melted)


## Allele level

In [ ]:
combined_Tstats_allele = pd.merge(PD_baseline_Tstats_allele, PD_habituation_Tstats_allele, on =['dataset'], how ='left')
combined_Tstats_allele=combined_Tstats_allele.sort_index()
# PD_combined_Tstats=PD_combined_Tstats.drop(index="N2")
combined_Tstats=combined_Tstats_allele.dropna()
print(combined_Tstats_allele)
combined_Tstats_normalize_allele=(combined_Tstats_allele-combined_Tstats_allele.mean())/combined_Tstats_allele.std()
print(combined_Tstats_normalize_allele)
print(combined_Tstats_normalize_allele[combined_Tstats_normalize_allele.index=="N2"])
print(combined_Tstats_normalize_allele.index)
combined_Tstats_normalize_allele_2=combined_Tstats_normalize_allele-combined_Tstats_normalize_allele[combined_Tstats_normalize_allele.index=="N2"].squeeze()
print(combined_Tstats_normalize_allele_2[combined_Tstats_normalize_allele_2.index=="N2"])

first_col=combined_Tstats_allele.pop("Area")
combined_Tstats_allele.insert(0,"Area",first_col)

first_col=combined_Tstats_allele.pop("Length")
combined_Tstats_allele.insert(0,"Length",first_col)

first_col=combined_Tstats_allele.pop("Width")
combined_Tstats_allele.insert(0,"Width",first_col)

first_col=combined_Tstats_allele.pop("Angular Speed")
combined_Tstats_allele.insert(5,"Angular Speed",first_col)

print(combined_Tstats_allele)

combined_Tstats_allele=combined_Tstats_allele.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                             "Habituation of Probability": "Habituation of Respones Probability",
                                                             "Habituation of Speed":"Habituation of Response Speed",
                                                             "Initial Duration": "Initial Response Duration",
                                                             "Initial Probability": "Initial Response Probability",
                                                             "Initial Speed": "Initial Response Speed",
                                                             "Final Duration": "Final Response Duration",
                                                             "Final Probability": "Final Response Probability",
                                                             "Final Speed": "Final Response Speed",
                                                             "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                             "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                             "Recovery Speed": "Spontaneous Recovery of Response Speed"})

combined_Tstats_normalize_allele_2=combined_Tstats_normalize_allele_2.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                                                             "Habituation of Probability": "Habituation of Respones Probability",
                                                                                             "Habituation of Speed":"Habituation of Response Speed",
                                                                                             "Initial Duration": "Initial Response Duration",
                                                                                             "Initial Probability": "Initial Response Probability",
                                                                                             "Initial Speed": "Initial Response Speed",
                                                                                             "Final Duration": "Final Response Duration",
                                                                                             "Final Probability": "Final Response Probability",
                                                                                             "Final Speed": "Final Response Speed",
                                                                                             "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                                                             "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                                                             "Recovery Speed": "Spontaneous Recovery of Response Speed"})
print(combined_Tstats_normalize_allele_2)

first_col=combined_Tstats_normalize_allele_2.pop("Area")
combined_Tstats_normalize_allele_2.insert(0,"Area",first_col)

first_col=combined_Tstats_normalize_allele_2.pop("Length")
combined_Tstats_normalize_allele_2.insert(0,"Length",first_col)

first_col=combined_Tstats_normalize_allele_2.pop("Width")
combined_Tstats_normalize_allele_2.insert(0,"Width",first_col)

first_col=combined_Tstats_normalize_allele_2.pop("Angular Speed")
combined_Tstats_normalize_allele_2.insert(5,"Angular Speed",first_col)

last_col=combined_Tstats_normalize_allele_2.pop("Spontaneous Recovery of Response Duration")
combined_Tstats_normalize_allele_2.insert(22,"Spontaneous Recovery of Response Duration",last_col)

last_col=combined_Tstats_normalize_allele_2.pop("Spontaneous Recovery of Response Probability")
combined_Tstats_normalize_allele_2.insert(22,"Spontaneous Recovery of Response Probability",last_col)

last_col=combined_Tstats_normalize_allele_2.pop("Spontaneous Recovery of Response Speed")
combined_Tstats_normalize_allele_2.insert(22,"Spontaneous Recovery of Response Speed",last_col)

print(combined_Tstats_normalize_allele_2)

combined_Tstats_normalized_melted_allele=combined_Tstats_normalize_allele_2.reset_index()
combined_Tstats_normalized_melted_allele=pd.melt(combined_Tstats_normalized_melted_allele, id_vars=['dataset'],
                                                   var_name='Metric',
                                                   value_name='T_score')
print(combined_Tstats_normalized_melted_allele)


combined_Tstats_melted_allele=combined_Tstats_allele.reset_index()
combined_Tstats_melted_allele=pd.melt(combined_Tstats_melted_allele, id_vars=['dataset'],
                                       var_name='Metric',
                                       value_name='T_score')
print(combined_Tstats_melted_allele)
combined_Tstats_melted_allele_sorted=combined_Tstats_melted_allele.sort_values(by=['T_score'])
print(combined_Tstats_melted_allele_sorted)

## sqlite3 upload data

## A janky way to add data and update the sql 

1. Read table to pd.DataFrame
2. Add new data to pd.DataFrame
3. Replace old table with newly updated pd.DataFrame

In [ ]:
def read(table):
    result = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    return result

conn = sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
old_tap_output = read('tap_response_data')
old_tap_tstat_gene = read('tstat_gene_data')
old_tap_tstat_allele = ('tstat_allele_data')
old_gene_profile_data = read('gene_profile_data')
old_allele_profile_data = read('allele_profile_data')
old_gene_MSD = read('gene_MSD')
old_allele_MSD = read('allele_MSD')

conn.close()

In [16]:
import sqlite3
conn = sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
# cur.execute("DROP TABLE IF EXISTS tap_baseline_data")
# cur.execute("DROP TABLE IF EXISTS allele_phenotype_data")
# cur.execute("DROP TABLE IF EXISTS tstat_data")

# print("-"*60)

# cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
# cur.execute(".tables")
conn.close()

[('tstat_gene_data',), ('tstat_allele_data',), ('gene_profile_data',), ('allele_profile_data',), ('tap_response_data',), ('gene_MSD',), ('allele_MSD',)]
[]


In [ ]:
'''
input: (pd.DataFrame, pd.Dataframe or None, str)
    1 or 2 pd.DataFrames with the same indexes (columns), and sql table name (str)
    first DataFrame is old DataFrame read from the .db file
    second DataFrame is new processed DataFrame to be appended to the old one
    

function: If only one DataFrame, replace the .db tabl with this one DataFrame
 Otherwise (when given two dataframes):
     1. concatenate new DataFrame to old DataFrame
     2. to_sql, if_exists = 'replace'

'''

def update_table(df1, df2=None, table_name):
    if df2=None:
        df1.to_sql(table_name, conn, if_exists='replace', index=False)
    else:
        new_df=pd.concat([df1, df2], ignore_index=True)
        new_df.to_sql(table_name, conn, if_exists='replace', index=False)
        

In [ ]:
tap_output
combined_Tstats_normalize_2
combined_Tstats_normalize_allele_2
combined_Tstats_normalized_melted
combined_Tstats_normalized_melted_allele
combined_MSD
allele_combined_MSD

In [93]:
conn=sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
tap_output.to_sql('tap_response_data', conn, if_exists='replace', index=False)
# baseline_output.to_sql('tap_baseline_data', conn, if_exists='replace', index=False)
combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='replace', index=False)
combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='replace', index=False)
combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='replace', index=False)
combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='replace', index=False)
combined_MSD.to_sql('gene_MSD', conn, if_exists='replace', index=False)
allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='replace', index=False)
# combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)
print(conn.total_changes)
conn.close()

# Want to test edge cases of pd.to_sql functionality#############

91


In [ ]:
print(conn.total_changes)

In [ ]:
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM table_name WHERE primary_key_column = ?", (value,))
exists = cursor.fetchone()[0] > 0

if exists:
    print("Primary key already exists in the table.")
else:
    print("Primary key does not exist in the table.")

In [ ]:
conn.close()

# The below cells are from Federico's example to set up a sqlite3 database


In [ ]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


In [ ]:
# Function for basic data validation
def validate_data(df):
    # Define expected data types
    expected_dtypes = {
        'Unnamed: 0': 'int64',
        'time': 'float64',
        'dura': 'float64',
        'dist': 'float64',
        'prob': 'float64',
        'speed': 'float64',
        'plate': 'int64',
        'taps': 'float64',
        'dataset': 'object',
        'Gene': 'object',
        'Allele': 'object'
    }
    
    # Type validation
    for col, expected_dtype in expected_dtypes.items():
        if df[col].dtype != expected_dtype:
            return False, f"Column {col} has incorrect data type {df[col].dtype}. Expected {expected_dtype}."
    
    # Range validation for specific columns
    if df['prob'].min() < 0 or df['prob'].max() > 1:
        return False, "Column 'prob' has values outside the range [0, 1]."
    
    return True, "Data is valid."

In [ ]:
is_valid, validation_message = validate_data(df)
if is_valid:
    # Create SQLite database and insert data
    try:
        conn = sqlite3.connect('c_elegans_data.db')
        df.to_sql('experiment_data', conn, if_exists='replace', index=False)
        etl_status = "Data successfully inserted into SQLite database."
    except IntegrityError as e:
        etl_status = f"Data insertion failed: {e}"
else:
    etl_status = f"Data validation failed: {validation_message}"

print(etl_status)

## PCA Playtest

The below cells are from the Jeremy Seamans Lecture exploring PCA. 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca.transform(X)

In [ ]:
pca=PCA()
pca.fit(X)


In [ ]:
components=pca.transform(X)

In [ ]:
conn.close()

In [ ]:
print(components)

In [ ]:
import plotly.express as px

features = ["Instantaneous Speed", "Interval Speed", "Bias", "Width","Length"]

pca = PCA()
components = pca.fit_transform(X[features])

labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,width=800, height=800,
    labels=labels,
    dimensions=range(4))
    # color=df["species"]
# fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
print(X)